In [1]:
import requests
import pandas as pd
import json
import os
import re
import html

from dotenv import load_dotenv
from neo4j import GraphDatabase


load_dotenv()


NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_PASS = os.getenv('NEO4J_PASS')
neo4j_client = GraphDatabase.driver(NEO4J_URI, auth=('neo4j', NEO4J_PASS))
retrieval = "InitialIngest"

def execute_query(query, params=None, neo4j=neo4j_client):
    with neo4j_client.session() as session:
        result = session.run(query, parameters=params)
        # Convert the result to a DataFrame
        records = [record.data() for record in result]
        return pd.DataFrame(records)


In [ ]:
##
def call_snapshot_api(query, snapshot_url="https://hub.snapshot.org/graphql", counter=0):
    if counter > 10:
        raise Exception("Max retries exceeded.")
    response = requests.post(snapshot_url, json={"query": query})
    if response.status_code == 504:
        # Handle Gateway Timeout
        return make_api_call(query, snapshot_url, counter=counter + 1)
    data = response.json()
    if "errors" in data:
        raise Exception("Error in API response: {}".format(data["errors"]))
    return data["data"]



def clean_text(text):
    # Unescape HTML entities
    text = html.unescape(text)
    
    # Remove HTML tags
    text = re.sub(r'<[^>]+>', '', text)
    
    # Replace smart double quotes with standard quotes
    text = re.sub(r'[“”]', '"', text)
    
    # Replace smart single quotes with standard quotes
    text = re.sub(r"[‘’]", "'", text)
    
    # Replace long dashes with hyphens
    text = re.sub(r'[—–]', '-', text)
    
    # Remove non-ASCII characters
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    
    # Replace other non-standard quote patterns (if any specific patterns are known they should be added here)
    # Example: text = re.sub(r'weird_pattern', 'replacement', text)
    
    return text


In [ ]:
arb_space_id = "arbitrumfoundation.eth"  # Replace with the actual space ID

spaces_query = f"""
{{
    space(id: "{arb_space_id}") {{
        id
        name
        about
        avatar
        website
        twitter
        github
        network
        symbol
        strategies {{
            name
            params
        }}
        admins
        members
    }}
}}
""" 

proposals_query = f"""
{{
    proposals(
        where: {{
            space_in: ["{arb_space_id}"]
        }},
        first: 1000
    ) {{
        id
        title
        body
        choices
        start
        end
        state
        author
    }}
}}
""" 

# votes_query = f"""
# {{
#     votes(
#         where: {{
#             proposal: "{arb_space_id}"
#         }},
#         first: 10
#     ) {{
#         id
#         voter
#         choice
#     }}
# }}
# """

In [ ]:
space_data = call_snapshot_api(spaces_query)
print(json.dumps(space_data, indent=4))


In [ ]:
## create snapshot space

params = {
    'spaceId': space_data['space']['id'],
    'name': space_data['space']['name'],
    'website': space_data['space']['website'],
    'twitter': space_data['space']['twitter'],
    'strategyToken': space_data['space']['strategies'][0]['params']['address']
}

space_query = """
    match (entity:Entity {name: "Arbitrum Foundation"}) 
    with entity
    merge (space:Snapshot:Space {spaceId: $spaceId}) 
    set space.name = $name
    set space.website = $website
    set space.twitter = $twitter 
    set space.strategyToken = $strategyToken
    with entity, space 
    merge (entity)-[r:ACCOUNT]->(space)
    return entity, r, space
"""

execute_query(space_query, params)

In [ ]:
# dump

In [ ]:
proposals_data = call_snapshot_api(proposals_query)

proposals_data['proposals'][0].keys()

In [ ]:
proposals_data_df = pd.DataFrame(proposals_data['proposals'])

In [ ]:
proposals_data_df.to_csv("snapshot-scraped-data/arb-snapshot-proposals-20240303.csv")

In [ ]:
counter = 0 
total = len(proposals_data['proposals'])

## create proposals
for i in proposals_data['proposals']:
    counter += 1
    print(f"Ingesting proposal {str(counter)} out of {str(total)}...")
    
    name = clean_text(i['title'])
    text = clean_text(i['body'])
    authorAddress = i['author'].lower()
    props_params = {
        'id': i['id'],
        'name': i['title'], 
        'startDt': i['start'],
        'endDt': i['end'],
        'choices': i['choices'], 
        'name': name, 
        'text': text,
        'authorAddress': authorAddress
    }
    props_query = """
    merge (prop:Snapshot:Proposal {id: $id})
    set prop.name = $name
    set prop.text = $text
    set prop.startDt = $startDt
    set prop.endDt = $endDt
    set prop.choices = $choices
    """
    execute_query(props_query, props_params)

In [ ]:
### connect

execute_query("""match (prop:Proposal) match (space:Space)
with prop, space
merge (space)-[r:PROPOSAL]->(proposal)""")



In [ ]:
## create and link authors
counter = 0 
total = len(proposals_data['proposals'])

## create proposals
for i in proposals_data['proposals']:
    counter += 1
    print(f"Ingesting record {str(counter)} out of {str(total)}...")
    
    authorAddress = i['author'].lower()

    walletParams = {
        'authorAddress' : authorAddress,
        'proposalId': i['id']
    }

    walletsQuery = """
    merge (wallet:Wallet {address: $authorAddress}) 
    with wallet 
    match (proposal:Proposal:Snapshot {id: $proposalId}) 
    with wallet, proposal 
    merge (wallet)-[r:AUTHOR]->(proposal)
    """
    execute_query(walletsQuery, walletParams)

In [ ]:
## cool voters

In [ ]:
def get_all_votes_for_proposals(proposals_response):
    votes_query_template = """
    {{
        votes (
            first: 1000,
            skip: {skip},
            where: {{
                proposal_in: {proposal_ids}
            }}
        ) {{
            id
            voter
            choice
            proposal {{
                id
            }}
        }}
    }}
    """
    proposal_ids = [proposal["id"] for proposal in proposals_response['proposals']]
    all_votes = []
    for proposal_id in proposal_ids:
        skip = 0
        while True:
            query = votes_query_template.format(skip=skip, proposal_ids=json.dumps([proposal_id]))
            response = requests.post("https://hub.snapshot.org/graphql", json={"query": query})
            if response.status_code == 200:
                data = response.json()["data"]["votes"]
                if not data:
                    break
                all_votes.extend(data)
                skip += 1000
            else:
                break
    return all_votes

all_votes = get_all_votes_for_proposals(proposals_data)

In [ ]:
len(all_votes)

In [ ]:
proposal_votes_df = pd.DataFrame(all_votes)
type(proposal_votes_df.iloc[0]['proposal'])
proposal_votes_df['proposalId'] = proposal_votes_df['proposal'].apply(lambda x: x['id'])
proposal_votes_df['voter'] = proposal_votes_df['voter'].apply(lambda x: x['voter'].lower())

In [ ]:
proposal_votes_df_upload = proposal_votes_df[['voter', 'proposalId']]
proposal_votes_df_upload.to_csv("snapshot-scraped-data/arb-snapshot-votes-initial.csv")

In [ ]:
voters_create = """
LOAD CSV WITH HEADERS FROM 'https://github.com/jchanolm/arbitrum-data/blob/main/notebooks/arb-data-pipelines/snapshot/snapshot-scraped-data/arb-snapshot-votes-initial-fixed.csv' AS row
MERGE (w:Wallet {address: row.voter})
"""

execute_query(voters)

In [ ]:
voters_connect = """
LOAD CSV WITH HEADERS FROM 'https://github.com/jchanolm/arbitrum-data/blob/main/notebooks/arb-data-pipelines/snapshot/snapshot-scraped-data/arb-snapshot-votes-initial-fixed.csv' AS row
MATCH (w:Wallet {address: row.voter})
MATCH (prop:Proposal {id: row.proposalId})
MERGE (w)-[r:VOTED]->(prop)
"""

execute_query(voters_connect)

In [ ]:
### set choice

In [ ]:
connect_voters_to_proposals = """
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/jchanolm/arbitrum-data/main/notebooks/arb-governance-data/snapshot/snapshot-scraped-data/cleaned_arb_snapshot_votes.csv' AS row
MATCH (w:Wallet {address: tolower(row.voter)})
MATCH (p:Proposal {id: row.proposalId})
MERGE (w)-[:VOTED {choice: row.choice}]->(p)
"""

execute_query(connect_voters_to_proposals)

In [ ]:
## connect snapshot urls from grant subs

connect_snapshot_proposals_to_grantees = """
connect
match (prop:Snapshot:Proposal)
match (grantee:Grantee)-[r:GRANTEE]-(grant:GrantInitiative)
where r.grantApprovalAction contains prop.id 
with prop, grantee, grant
merge (grant)-[r:PROPROSAL]->(prop)
merge (prop)-[r1:APPROVED_FUNDING]->(grantee)"""

execute_query(connect_snapshot_proposals_to_grantees)